# Getting started with Determined, the open-source deep learning training platform - Lab 4
## Hyperparameter Optimization with Determined

Next, let's run an experiment with the same model code, but this time leveraging Determined's hyperparameter tuning (aka Hyperparameter Optimization or **HPO**) to improve the model accuracy and let you find the best combination of hyperparameters for your particular model. 

ML engineers typically use HPO to efficiently determine the hyperparameter values that yield the best-performing model. Here in this lab, the hyperparameters in the experiment configuration file are specified as ranges instead of fixed values, and the `adaptive_asha` searcher method is used to explore the hyperparameter space.

**With HPO, an experiment consists of multiple training tasks (trials)**. All of the trials are training the model on the same dataset and code for the DL model. However each trial uses a different configuration of hyperparameters. 

For this part of lab, the number of trials to run, the set of user-defined hyperparameters range, the searcher method, and the amount of data (batches or epochs) on which to train the model are defined in the experiment configuration file _adaptive.yaml_.

>Note: The **searcher** is a method (an algorithm) that is used to find effective hyperparameter settings within a predifined range of hyperparameter values.

More about Hyperparameter optimization and Searcher methods supported by Determined can be found [here](https://docs.determined.ai/latest/training-hyperparameter/index.html#hyperparameter-tuning)

### 1- Create an experiment to train multiple models as part of a hyperparameter search, using Determined hyperparameter optimization (HPO)

Let's run an experiment with the same model definition (same code), but this time leveraging Determined's hyperparameter optimization functionality using the _adaptive.yaml_ experiment configuration file. 

#### Let's take a closer look at the experiment configuration file for HPO:

In [ ]:
!cat ~/source_control/Code/adaptive.yaml

----- to be continued from here ------

As you can see here, this configuration file is very similar to the _const.yaml_ file you used earlier. All you need to do to start a multi-GPU training workload (trial) is to specify the desired number of GPUs you want to use in the experiment configuration file and Determined takes care of the rest. For example:

                                      resources:
                                          slots_per_trial: 2

With this configuration, each trial within an experiment will use **2 GPUs** to train a single model, whether leveraging 2 GPUs on a single machine or 2 GPUs across multiple machines in the Kubernetes cluster.

#### Next, submit the experiment with the experiment configuration file _distributed.yaml_:

In [ ]:
masterUrl=!kubectl describe service determined-master-service-stagingdetai -n determinedai | grep gateway/8080 | awk '{print $3}'
det_master = str(masterUrl)[2:-2] # we remove any potential brackets
determined_master = "http://" + det_master
#
# Launch experiment to train the model with hyperparameter tuning
!~/.local/bin/det -m {determined_master} experiment create ~/source_control/Code/adaptive.yaml ~/source_control/Code

In the lab environment, the Kubernetes worker hosts have one GPU only. The training task (trial) needs 2 GPUs as per the experiment configuration file. Therefore, Determined Master brings up two PODs, for the same trial, on two different Kubernetes worker hosts.   

#### Using the command below, you will see that Determined Master has launched **two** PODs for the training task in the Kubernetes cluster with name in the form:

 _exp-\<experimentID\>-trial-\<TriaID\>-\<unique-name\>_

> Notice the Trial ID is the same for the two PODs, which means your experiment features a single trial with a fixed set of hyperparameters. 

In [ ]:
!kubectl get pods -n determinedai

#### Run the code cell below to monitor the execution progress of the experiment.

In [ ]:
!~/.local/bin/det -m {determined_master} experiment list | tail -1
# Get the experiment Id, remove spaces
myexpId=!~/.local/bin/det -m {determined_master} experiment list | tail -1 | cut -d'|' -f 1 |  tr -d ' '
# remove the trailer characters
myexpId=str(myexpId)[2:-2]
!~/.local/bin/det -m {determined_master} experiment describe {myexpId} --json | jq .[0].state

### 2- Monitor and visualize your experiment in Determined Web User Interface

To monitor the progress of the training task and access information on both training and validation performance, simply return to the Determined **WebUI**.

##### From the **Dashboard**, select the most recent experiment.

You should see the experiment as an active state. The graph is showing the training accuracy metric. You can see the graph changing in real time as the experiment runs.

From the **Metrics** menu, under **Training Metrics**, select _categorical_accuracy_ (see picture below for an example). This metric indicates the model accuracy on training data while the _val_categorical_accuracy_ indicates the model accuracy on validation data.

After the experiment completes, you can see on the experiment detail page that training the model with the hyperparameter settings in `distributed.yaml` yields a validation accuracy between 93% and 97%. 

Scroll down to see a list of training validation workloads and their metrics for the metric types you previously selected. 
You might see one or two validation workloads with checkpoints. By default, Determined will checkpoint the most recent and the best model per training task (trial). If the most recent checkpoint is also the best checkpoint for a given trial, only one checkpoint will be saved for that trial as shown in the picture below.

<img src="WebUI-Exp-distribute-graph.png" height="520" width="900">

### 3 - TensorBoard visualization

You can also use [TensorBoard](https://www.tensorflow.org/tensorboard) for visualizing and inspecting the deep learning models. 

#### Run the code cell below to launch the TensorBoard server instance.

This may take a minute or so as Determined has to launch the Tensorboard server as a Kubernetes POD. 

In [ ]:
print (f"Start a Tensordboard server instance for your Experiment {myexpId} with TensorBoard instance ID:")
# start the tensorBoard server instance for the experiment
!~/.local/bin/det -m {determined_master} tensorboard start -d {myexpId}

#### Run the code cell below to get the Tensorboard URL for your experiment. Then, click on the link to connect.

>**Note:** The associated TensorBoard server is launched as a container POD in the Kubernetes cluster. Determined proxies HTTP requests to and from the TensorBoard container through the Determined Master node.

In [ ]:
mytensorboard=!~/.local/bin/det -m {determined_master} tensorboard list | grep RUNNING | cut -d'|' -f 1 |  tr -d ' '
mytensorboard=str(mytensorboard)[2:-2]
#print (f"{mytensorboard}")
print (f"Your tensorboard is running at http://notebooks.hpedev.io:{portUI}/proxy/{mytensorboard}/")
print (f"Click on the link to connect.")

<img src="TensorBoard-distribute-graph.png" height="413" width="900">

Determined created TensorBoard plots to show the training loss, validation loss, training accuracy and validation accuracy for the training task (trial).

#### When you have finished with Tensorboard, run the code cell below to `kill` the Tensorboard process

In [ ]:
!~/.local/bin/det -m {determined_master} tensorboard kill {mytensorboard}

### 4 - List the best model created by the training process
By default, Determined will save the most recent and the best checkpoint per training task (trial) according to the validation metrics specified in the Searcher section of the configuration file for the experiment.

* _det experiment list-checkpoints [--best] [N best checkpoints to return] \<experiment_Id\>_

>**Note**: Upon completion of the training task, if the most recent checkpoint is also the best checkpoint for a given trial, only one checkpoint will be saved for that trial by Determined. Otherwise, two checkpoints will be saved. Other checkpoints will be automatically deleted to reclaim space.

#### Run the code cell below to display the best checkpoint for your experiment

In [ ]:
#list the best Trial checkpoint(s) (training task):
!~/.local/bin/det -m {determined_master} experiment list-checkpoints --best 1 {myexpId}

### 5- Delete the checkpoints to reclaim some storage space in the storage file system

The default **checkpoint garbage collection policy** dictates Determined to save the most recent and the best checkpoint per training task (trial). The ***save_experiment_best***, ***save_trial_best*** and ***save_trial_latest*** parameters specify which checkpoints to save. The default policy is set as follows:

  * save_experiment_best:0 
  * save_trial_best:1
  * save_trial_latest:1
 
#### Run the code cell below to reclaim some storage disk space by changing the default checkpoint garbage collection policy as shown below:

In [ ]:
# Delete the checkpoints data for the distributed training
!~/.local/bin/det -m {determined_master} experiment set gc-policy --yes --save-experiment-best 0 --save-trial-best 0 --save-trial-latest 0 {myexpId}

#### Now, logout from your local Jupyter Notebook and go back to the JupyterHub notebook to perform some cleanup and for a wrap up of the workshop.